In [ ]:
#1) MOUNT THE GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#2) ACCESS THE DATASET

import os

address='/content/drive/MyDrive/datasets/PlantVillageMini'

#List Files
print(os.listdir(address))
num_classes=len(os.listdir(address))
print(num_classes)

count=0
for i in os.listdir(address):
  for j in os.listdir(os.path.join(address,i)):
    count+=1

print("Number of images:",count)

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight']
5
Number of images: 4627


In [ ]:
#3) CREATING A TENSORFLOW DATASET USING THE IMAGES

#import os
import numpy as np
from sklearn.model_selection import train_test_split  #for train test split
from tensorflow.keras.utils import to_categorical #For one hot encoding

data_dir=address   #Root folder with class subfolders
class_names=os.listdir(data_dir)
num_classes=len(class_names)

#Collect file paths and labels
file_paths=[]
labels=[]

for class_id, class_name in enumerate(class_names):
  for image_name in os.listdir(os.path.join(address,class_name)):
    file_paths.append(os.path.join(address,class_name,image_name))
    labels.append(class_id)     #class_id is numerical 0,1,2,3,...

#Convert it into numpy arrays
file_paths=np.array(file_paths)
labels=np.array(labels)

print(file_paths.shape)
print(labels.shape)

#Train test split
X_train,X_test,y_train,y_test=train_test_split(file_paths,labels,test_size=0.2,random_state=42)

#one hot encoding
y_train=to_categorical(y_train,num_classes)
y_test=to_categorical(y_test,num_classes)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4627,)
(4627,)
(3701,)
(926,)
(3701, 5)
(926, 5)


In [ ]:
#4) LOADING AND PREPROCESSING THE IMAGES

import tensorflow as tf

print("Num GPU Available: ",len(tf.config.list_physical_devices('GPU')))
print("Num GPU Available: ",len(tf.config.list_physical_devices('CPU')))

input_size=[224,224]

def preprocess_image(image_path, label):
  image=tf.io.read_file(image_path)
  image=tf.image.decode_jpeg(image, channels=3)   #read the jpeg image as color
  image=tf.image.resize(image, input_size)       #Resized the image to standard size
  image=image/255.0    #normalize the images
  return image,label

#Creating a tensorflow dataset
train_dataset=tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset=tf.data.Dataset.from_tensor_slices((X_test, y_test))

#apply preprocessing
train_dataset=train_dataset.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)   #batches of 32, with prefetch optimization for faster load
test_dataset=test_dataset.map(preprocess_image).batch(32)

Num GPU Available:  0
Num GPU Available:  1


In [ ]:
#5) LOADING THE PREPROCESSED MODEL

from tensorflow.keras import layers, models

#load the ResNetV2
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(224,224,3),
    include_top=False,    #Removing the last layer
    weights='imagenet'    #load weights trained on imagenet
)

#freeze the base model
base_model.trainable=False    #weights of original model will not change

#adding our custom classification layer
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),    #reduces feature map to a vector
    layers.Dense(128, activation="relu"),   #extra dense layer for better training
    layers.Dense(num_classes, activation="softmax")
])

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,827,717 (90.90 MB)

 Trainable params: 262,917 (1.00 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

In [ ]:
#6) COMPILE THE MODEL

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

EPOCHS=10
#early_stop=EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')   #stop training early if accuracy starts to reduce
reduce_lr=ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-6)

model.fit(train_dataset, validation_data=test_dataset, epochs=EPOCHS, callbacks=[reduce_lr])

Epoch 1/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 873s 7s/step - accuracy: 0.7859 - loss: 0.5792 - val_accuracy: 0.9395 - val_loss: 0.1602 - learning_rate: 0.0010
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 921s 7s/step - accuracy: 0.9597 - loss: 0.1125 - val_accuracy: 0.9611 - val_loss: 0.1075 - learning_rate: 0.0010
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 929s 8s/step - accuracy: 0.9797 - loss: 0.0632 - val_accuracy: 0.9525 - val_loss: 0.1221 - learning_rate: 0.0010
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9829 - loss: 0.0495
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
116/116 ━━━━━━━━━━━━━━━━━━━━ 877s 8s/step - accuracy: 0.9829 - loss: 0.0495 - val_accuracy: 0.9341 - val_loss: 0.1787 - learning_rate: 0.0010
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 854s 7s/step - accuracy: 0.9882 - loss: 0.0359 - val_accuracy: 0.9806 - val_loss: 0.0513 - learning_rate: 5.0000e-04
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 876s 8s/step - accuracy: 0.9940 - loss: 0

In [ ]:
#Testing for accuracy
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

29/29 ━━━━━━━━━━━━━━━━━━━━ 165s 6s/step - accuracy: 0.9832 - loss: 0.0381
Test Loss: 0.04789106920361519
Test Accuracy: 98.27%


In [ ]:
model.save("LeafClassifier_ResNet50_1.keras")
from google.colab import files
files.download('LeafClassifier_ResNet50_1.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

USING RESNETV2 WITHOUT EXTRA LAYERS

In [ ]:
#5) LOADING THE PREPROCESSED MODEL

from tensorflow.keras import layers, models


#load the ResNetV2
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(224,224,3),
    include_top=False,    #Removing the last layer
    weights='imagenet'    #load weights trained on imagenet
)

#freeze base model
for layer in base_model.layers:
    layer.trainable = False

#adding our custom classification layer
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),    #reduces feature map to a vector
    layers.Dense(num_classes, activation="softmax")
])

#unfreeze last two layers
for layer in base_model.layers[-2:]:
    layer.trainable = True

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │        10,245 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,575,045 (89.93 MB)

 Trainable params: 14,341 (56.02 KB)

 Non-trainable params: 23,560,704 (89.88 MB)

In [ ]:
#6) COMPILE THE MODEL

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

EPOCHS=10
#early_stop=EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')   #stop training early if accuracy starts to reduce
reduce_lr=ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-6)

model.fit(train_dataset, validation_data=test_dataset, epochs=EPOCHS, callbacks=[reduce_lr])

Epoch 1/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 1404s 12s/step - accuracy: 0.7198 - loss: 0.8010 - val_accuracy: 0.9330 - val_loss: 0.2341 - learning_rate: 0.0010
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 873s 7s/step - accuracy: 0.9316 - loss: 0.2329 - val_accuracy: 0.9611 - val_loss: 0.1547 - learning_rate: 0.0010
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 846s 7s/step - accuracy: 0.9546 - loss: 0.1636 - val_accuracy: 0.9665 - val_loss: 0.1240 - learning_rate: 0.0010
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 824s 7s/step - accuracy: 0.9659 - loss: 0.1255 - val_accuracy: 0.9687 - val_loss: 0.1060 - learning_rate: 0.0010
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 909s 7s/step - accuracy: 0.9759 - loss: 0.1000 - val_accuracy: 0.9730 - val_loss: 0.0945 - learning_rate: 0.0010
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 850s 7s/step - accuracy: 0.9837 - loss: 0.0819 - val_accuracy: 0.9762 - val_loss: 0.0862 - learning_rate: 0.0010
Epoch 7/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 864s 7s/step - accuracy: 0.9885 - loss: 

In [ ]:
#Testing for accuracy
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

29/29 ━━━━━━━━━━━━━━━━━━━━ 159s 6s/step - accuracy: 0.9765 - loss: 0.0614
Test Loss: 0.06699324399232864
Test Accuracy: 97.62%


In [ ]:
model.save("LeafClassifier_ResNet50_2.keras")
from google.colab import files
files.download('LeafClassifier_ResNet50_2.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

LOADING RESNET50 WITH UNFREEZED TOP LAYERS + EXTRA LAYERS

In [ ]:
#5) LOADING THE PREPROCESSED MODEL

from tensorflow.keras import layers, models


#load the ResNetV2
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(224,224,3),
    include_top=False,    #Removing the last layer
    weights='imagenet'    #load weights trained on imagenet
)

#freeze base model
for layer in base_model.layers:
    layer.trainable = False

#adding our custom classification layer
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),    #reduces feature map to a vector
    layers.Dense(128, activation="relu"),   #extra dense layer for better training
    layers.Dense(num_classes, activation="softmax")
])

#unfreeze last two layers
for layer in base_model.layers[-2:]:
    layer.trainable = True

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,827,717 (90.90 MB)

 Trainable params: 267,013 (1.02 MB)

 Non-trainable params: 23,560,704 (89.88 MB)

In [ ]:
#6) COMPILE THE MODEL

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

EPOCHS=10
#early_stop=EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')   #stop training early if accuracy starts to reduce
reduce_lr=ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-6)

model.fit(train_dataset, validation_data=test_dataset, epochs=EPOCHS, callbacks=[reduce_lr])

Epoch 1/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 858s 7s/step - accuracy: 0.8279 - loss: 0.5108 - val_accuracy: 0.9590 - val_loss: 0.1123 - learning_rate: 0.0010
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 816s 7s/step - accuracy: 0.9640 - loss: 0.1032 - val_accuracy: 0.9568 - val_loss: 0.1094 - learning_rate: 0.0010
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 903s 7s/step - accuracy: 0.9789 - loss: 0.0628 - val_accuracy: 0.9546 - val_loss: 0.1259 - learning_rate: 0.0010
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9848 - loss: 0.0427
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
116/116 ━━━━━━━━━━━━━━━━━━━━ 861s 7s/step - accuracy: 0.9848 - loss: 0.0427 - val_accuracy: 0.9374 - val_loss: 0.2163 - learning_rate: 0.0010
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 862s 7s/step - accuracy: 0.9871 - loss: 0.0332 - val_accuracy: 0.9795 - val_loss: 0.0533 - learning_rate: 5.0000e-04
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 806s 7s/step - accuracy: 0.9963 - loss: 0

In [ ]:
#Testing for accuracy
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

29/29 ━━━━━━━━━━━━━━━━━━━━ 160s 6s/step - accuracy: 0.9833 - loss: 0.0438
Test Loss: 0.04697323217988014
Test Accuracy: 98.27%


In [ ]:
model.save("LeafClassifier_ResNet50_3.keras")
from google.colab import files
files.download('LeafClassifier_ResNet50_3.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>